In [1]:
# %reload_ext autoreload
# %autoreload 1   
import numpy as np
import os
import argparse
import torch
import torchvision
import torchvision.transforms as transforms
import torch.utils.data as data
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from pytorch_models import LSTM
import imp
import torch.optim as optim

import matplotlib.pyplot as plt
import time
import ray
from ray import tune
from ray.tune import track
track.init()

from ray.tune.schedulers import AsyncHyperBandScheduler

seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

input_dim = 120
output_size = 5
hidden_dim = 1000
n_layers = 1

In [9]:
TRAIN_DATA_PATH = "/home/asalimi/Synths_Stacks_Search/feature_extraction/lstm_data/simple/train"
TEST_DATA_PATH = "/home/asalimi/Synths_Stacks_Search/feature_extraction/lstm_data/simple/test"
TRANSFORM_IMG = transforms.Compose(
    [transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(), 
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
def get_loaders(bs=16):
    train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=TRANSFORM_IMG)
    train_loader = data.DataLoader(train_data, batch_size=bs,shuffle=True, num_workers=4)
    test_data = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform=TRANSFORM_IMG)
    test_loader  = data.DataLoader(test_data, batch_size=bs, shuffle=True, num_workers=4)
    val_loader=test_loader
    return train_loader,test_loader,train_data,test_data,val_loader

train_loader,test_loader,train_data,test_data,val_loader = get_loaders(32)
classes=list(train_data.class_to_idx.keys()) #get list of classes
targets=[]
for batch_idx, (d, l) in enumerate(train_loader):
    targets.extend(list(l.numpy()))
# Compute samples weight (each sample should get its own weight)
target=torch.tensor(targets)
class_sample_count = torch.tensor(
    [(target == t).sum() for t in torch.unique(target, sorted=True)])
weight = 1. / class_sample_count.float()
samples_weight = torch.tensor([weight[t] for t in target])
print("here are the weights to account for class sizes")
print(dict(zip(classes,list(weight.numpy()*1000))))
CLASS_WEIGHTS =weight

here are the weights to account for class sizes
{'clap': 2.7027028, 'hat': 0.58377117, 'kick': 0.9910803, 'snare': 0.80775446, 'stacks': 0.2247191}


In [10]:
def train(model, optimizer, data_loader,  loss=nn.CrossEntropyLoss(),clip=5,device=torch.device("cpu")):
    loss_criteria=loss
    for batch_idx, (data, target) in enumerate(data_loader):
        data, target = data.to(device), target.to(device)
        h = model.init_hidden(data.size()[0],device)
        h = tuple([e.data for e in h])
        model.zero_grad()
        output, h = model(data, h)
        loss = loss_criteria(output, target)
        loss.backward()
        if clip!=0:
            nn.utils.clip_grad_norm_(model.parameters(),clip)
        optimizer.step()

def test(model, data_loader, loss=nn.CrossEntropyLoss(),device=torch.device("cpu")):
    loss_criteria=loss
    correct = 0
    total = 0
    val_losses = []
    with torch.no_grad():
        for d in data_loader:
            images, labels =  d[0].to(device), d[1].to(device)
            h = model.init_hidden(images.size()[0],device)
            outputs,h = model(images,h)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            #loss calculation (prolly don't need it)
            val_loss = loss_criteria(outputs.squeeze(), labels)
            val_losses.append(val_loss.item())
    return 100*(correct / total),np.mean(val_losses)
        
    
class Trainable(tune.Trainable):
    def _train(self):
        train(self.model, self.optimizer, self.train_loader, clip=self.clip, device=self.device)
        acc,val_loss = test(self.model, self.test_loader, device=self.device)
        return {"mean_accuracy": acc,"mean_loss":val_loss}
        
    def _setup(self,config):
        self.timestep = 0
        input_dim = 120
        seq_dim=100
        output_size = 5
        use_cuda = config.get("use_gpu") and torch.cuda.is_available()
        self.device = torch.device("cuda" if use_cuda else "cpu")
        self.model = LSTM(input_dim,seq_dim,config["hidden_size"],config["num_layers"],output_size,)
        self.model.to(self.device)
        self.train_loader,self.test_loader,_,_,_=get_loaders(bs=config["batch_size"])
        if config["optimizer"]=="adam":
            self.optimizer = torch.optim.Adam(self.model.parameters(), lr=config["lr"])
        elif config["optimizer"]=="adagrad":
            self.optimizer = torch.optim.Adagrad(self.model.parameters(), lr=config["lr"])
        elif config["optimizer"]=="rms":
            self.optimizer = torch.optim.RMSprop(self.model.parameters(), lr=config["lr"])
        elif config["optimizer"]=="adadelta":
            self.optimizer = torch.optim.Adadelta(self.model.parameters(), lr=config["lr"])
        self.clip=config["clip"]
        if config["weighted"]:
            self.loss_criteria=nn.CrossEntropyLoss()
        else:
            self.loss_criteria=nn.CrossEntropyLoss(CLASS_WEIGHTS)
    def _save(self, tmp_checkpoint_dir):
        checkpoint_path = os.path.join(tmp_checkpoint_dir, "model.pth")
        torch.save(self.model.state_dict(), checkpoint_path)
        return tmp_checkpoint_dir

    def _restore(self, tmp_checkpoint_dir):
        checkpoint_path = os.path.join(tmp_checkpoint_dir, "model.pth")
        self.model.load_state_dict(torch.load(checkpoint_path))


In [ ]:
sched = AsyncHyperBandScheduler(time_attr="time_total_s",
                                metric="mean_accuracy",max_t=900, 
                                grace_period=600, mode="max")
# sched=ray.tune.schedulers.MedianStoppingRule(
#                                 metric="mean_accuracy",min_time_slice=60,
#                                 grace_period=600, mode="max")

analysis = tune.run(
    Trainable,
    scheduler=sched,
    name="lstm-exp2",
    stop={
        "mean_accuracy": 90,
        "training_iteration": 100,
#         "validation_loss": 0.2
    },
    resources_per_trial={
        "cpu": 4,
        "gpu": 1
    },
    num_samples=1,
    checkpoint_at_end=True,
    checkpoint_freq=10,
    max_failures=2,
    config={
        "lr": tune.uniform(0.001, 0.1),
        "use_gpu": True,
        "batch_size": tune.grid_search([8,16,32,64]),
        "hidden_size": tune.grid_search([600,800,1400]),
        "num_layers": tune.grid_search([1,2]),
        "optimizer":tune.grid_search(["adam","rmsprop"]),
        "clip": tune.grid_search([1,5,10]),
        "weighted":tune.grid_search([True,False])

    })

2020-01-15 20:22:55,729	WARNING services.py:597 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-01-15 20:22:55,731	INFO resource_spec.py:216 -- Starting Ray with 5.52 GiB memory available for workers and up to 2.76 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-01-15 20:22:56,618	INFO ray_trial_executor.py:121 -- Trial Trainable_7d0c910c: Setting up new remote runner.


Trial name,status,loc,hidden_size,batch_size,optimizer,lr,num_layers,clip
Trainable_7d0c910c,RUNNING,,,,,,,
Trainable_7d0c910d,PENDING,,,,,,,
Trainable_7d0c910e,PENDING,,,,,,,
Trainable_7d0c910f,PENDING,,,,,,,
Trainable_7d0c9110,PENDING,,,,,,,
Trainable_7d0c9111,PENDING,,,,,,,
Trainable_7d0c9112,PENDING,,,,,,,
Trainable_7d0c9113,PENDING,,,,,,,
Trainable_7d0c9114,PENDING,,,,,,,
Trainable_7d0c9115,PENDING,,,,,,,


2020-01-15 20:22:56,877	INFO ray_trial_executor.py:121 -- Trial Trainable_7d0c910d: Setting up new remote runner.


Result for Trainable_7d0c910d:
  date: 2020-01-15_20-23-06
  done: false
  experiment_id: 0b58690229e748049416f3aa7a8bdccc
  experiment_tag: 1_batch_size=16,clip=1,hidden_size=400,lr=0.095121,num_layers=1,optimizer=adam
  hostname: calzone
  iterations_since_restore: 1
  mean_accuracy: 51.015697137580794
  mean_loss: 3.6972074613851658
  neg_mean_loss: -3.6972074613851658
  node_ip: 10.13.15.10
  pid: 19898
  time_since_restore: 6.415800333023071
  time_this_iter_s: 6.415800333023071
  time_total_s: 6.415800333023071
  timestamp: 1579144986
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 7d0c910d
  


Trial name,status,loc,hidden_size,batch_size,optimizer,lr,num_layers,clip,iter,total time (s),acc,loss
Trainable_7d0c910d,RUNNING,10.13.15.10:19898,400,16,adam,0.0951207,1,1,1,6.4158,51.0157,3.69721
Trainable_7d0c910c,RUNNING,,,,,,,,,,,
Trainable_7d0c910e,PENDING,,,,,,,,,,,
Trainable_7d0c910f,PENDING,,,,,,,,,,,
Trainable_7d0c9110,PENDING,,,,,,,,,,,
Trainable_7d0c9111,PENDING,,,,,,,,,,,
Trainable_7d0c9112,PENDING,,,,,,,,,,,
Trainable_7d0c9113,PENDING,,,,,,,,,,,
Trainable_7d0c9114,PENDING,,,,,,,,,,,
Trainable_7d0c9115,PENDING,,,,,,,,,,,


...

Result for Trainable_7d0c9110:
  date: 2020-01-15_20-47-14
  done: false
  experiment_id: 8210ebf1b2fb417d97bb68a0bf0eeab4
  experiment_tag: 4_batch_size=8,clip=5,hidden_size=400,lr=0.016446,num_layers=1,optimizer=adam
  hostname: calzone
  iterations_since_restore: 34
  mean_accuracy: 77.37765466297323
  mean_loss: 0.6619536696806055
  neg_mean_loss: -0.6619536696806055
  node_ip: 10.13.15.10
  pid: 19914
  time_since_restore: 385.51571583747864
  time_this_iter_s: 11.048609018325806
  time_total_s: 385.51571583747864
  timestamp: 1579146434
  timesteps_since_restore: 0
  training_iteration: 34
  trial_id: 7d0c9110
  


Trial name,status,loc,hidden_size,batch_size,optimizer,lr,num_layers,clip,iter,total time (s),acc,loss
Trainable_7d0c910f,TERMINATED,,400,64,adam,0.0602672,1,1,100,333.074,80.8864,0.546066
Trainable_7d0c910e,TERMINATED,,400,32,adam,0.0734674,1,1,100,413.805,51.0157,2.9961
Trainable_7d0c910d,TERMINATED,,400,16,adam,0.0951207,1,1,100,646.992,51.0157,3.91499
Trainable_7d0c910c,TERMINATED,,400,8,adam,0.0380795,1,1,69,902.659,76.639,0.723857
Trainable_7d0c9111,RUNNING,10.13.15.10:19899,400,16,adam,0.0164435,1,5,28,204.765,82.3176,0.542985
Trainable_7d0c9110,RUNNING,10.13.15.10:19914,400,8,adam,0.0164458,1,5,34,385.516,77.3777,0.661954
Trainable_7d0c9112,PENDING,,,,,,,,,,,
Trainable_7d0c9113,PENDING,,,,,,,,,,,
Trainable_7d0c9114,PENDING,,,,,,,,,,,
Trainable_7d0c9115,PENDING,,,,,,,,,,,


Result for Trainable_7d0c9111:
  date: 2020-01-15_20-47-15
  done: false
  experiment_id: 996f74f6c1f44b728ca0f605a7ec46e6
  experiment_tag: 5_batch_size=16,clip=5,hidden_size=400,lr=0.016443,num_layers=1,optimizer=adam
  hostname: calzone
  iterations_since_restore: 29
  mean_accuracy: 81.11726685133888
  mean_loss: 0.5339984657133326
  neg_mean_loss: -0.5339984657133326
  node_ip: 10.13.15.10
  pid: 19899
  time_since_restore: 212.02633047103882
  time_this_iter_s: 7.261148452758789
  time_total_s: 212.02633047103882
  timestamp: 1579146435
  timesteps_since_restore: 0
  training_iteration: 29
  trial_id: 7d0c9111
  
Result for Trainable_7d0c9111:
  date: 2020-01-15_20-47-23
  done: false
  experiment_id: 996f74f6c1f44b728ca0f605a7ec46e6
  experiment_tag: 5_batch_size=16,clip=5,hidden_size=400,lr=0.016443,num_layers=1,optimizer=adam
  hostname: calzone
  iterations_since_restore: 30
  mean_accuracy: 83.28716528162512
  mean_loss: 0.5144787640584743
  neg_mean_loss: -0.514478764058474

Trial name,status,loc,hidden_size,batch_size,optimizer,lr,num_layers,clip,iter,total time (s),acc,loss
Trainable_7d0c910f,TERMINATED,,400,64,adam,0.0602672,1,1,100,333.074,80.8864,0.546066
Trainable_7d0c910e,TERMINATED,,400,32,adam,0.0734674,1,1,100,413.805,51.0157,2.9961
Trainable_7d0c910d,TERMINATED,,400,16,adam,0.0951207,1,1,100,646.992,51.0157,3.91499
Trainable_7d0c910c,TERMINATED,,400,8,adam,0.0380795,1,1,69,902.659,76.639,0.723857
Trainable_7d0c9111,RUNNING,10.13.15.10:19899,400,16,adam,0.0164435,1,5,30,219.312,83.2872,0.514479
Trainable_7d0c9110,RUNNING,10.13.15.10:19914,400,8,adam,0.0164458,1,5,34,385.516,77.3777,0.661954
Trainable_7d0c9112,PENDING,,,,,,,,,,,
Trainable_7d0c9113,PENDING,,,,,,,,,,,
Trainable_7d0c9114,PENDING,,,,,,,,,,,
Trainable_7d0c9115,PENDING,,,,,,,,,,,


Result for Trainable_7d0c9110:
  date: 2020-01-15_20-47-26
  done: false
  experiment_id: 8210ebf1b2fb417d97bb68a0bf0eeab4
  experiment_tag: 4_batch_size=8,clip=5,hidden_size=400,lr=0.016446,num_layers=1,optimizer=adam
  hostname: calzone
  iterations_since_restore: 35
  mean_accuracy: 79.17820867959372
  mean_loss: 0.5871435443138724
  neg_mean_loss: -0.5871435443138724
  node_ip: 10.13.15.10
  pid: 19914
  time_since_restore: 396.71108531951904
  time_this_iter_s: 11.195369482040405
  time_total_s: 396.71108531951904
  timestamp: 1579146446
  timesteps_since_restore: 0
  training_iteration: 35
  trial_id: 7d0c9110
  


In [ ]:
analysis.get_best_config(metric="mean_accuracy")
analysis.dataframe()